# AWS Transcribe

[Documentation Boto3 to AWS Transcribe]()

### Installation

In [1]:
pip install boto3

Note: you may need to restart the kernel to use updated packages.


### Credentials

In [26]:
import os

os.environ['AWS_ACCESS_KEY_ID'] = ''
os.environ['AWS_SECRET_ACCESS_KEY'] = ''
os.environ['REGION_NAME'] = ''

### Transcribing an audio file

AWS Transcribe converts audio files containing speech into text. Let's transcribe an audio file.
Basically, the only function we need to use is *[client.start_transcription_job](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/transcribe/client/start_transcription_job.html)*, to which you pass the following arguments:

- TranscriptionJobName: The name of your job
- LanguageCode: The language code of the spoken language. Alternatively you can use "IdentifyLanguage"
- Media: S3 location of the file you want to transcribe
- OutputBucketName: [Not required] Name of the Output Bucket. Alternatively, aws creates one for you and you do not need to change access rights
- MediaFormat: Format of input file

After starting this job, we need to wait until its finished to download the results.

So let's create a new bucket, upload some mp3 file to it and obtain its text content!

In this notebook, we will walk through how to use boto3 to interact with 
[AWS Transcribe](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/transcribe.html) for converting speech into text.


In [27]:
import boto3
client = boto3.client('s3')
client.create_bucket(Bucket="demo-bucket-transcription-course")

{'ResponseMetadata': {'RequestId': '2D4NB6WSTQHSH7ZW',
  'HostId': 'EPouPGFV15pwJVZUzNXcRv2VLtGKXSuVQilPyjvEBcQLxc/3g/DPQ0tS0OiLyclTysjNCD1NTxo=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'EPouPGFV15pwJVZUzNXcRv2VLtGKXSuVQilPyjvEBcQLxc/3g/DPQ0tS0OiLyclTysjNCD1NTxo=',
   'x-amz-request-id': '2D4NB6WSTQHSH7ZW',
   'date': 'Wed, 17 Jan 2024 13:44:40 GMT',
   'location': '/demo-bucket-transcription-course',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'Location': '/demo-bucket-transcription-course'}

In [28]:
client.upload_file(Filename="./speech.mp3", Bucket="demo-bucket-transcription-course", Key="speech.mp3")

Now let's start the transcription job

In [29]:
transcribe_client = boto3.client('transcribe', region_name="us-east-1")
job_name = "TranscribeJob1"

transcribe_client.start_transcription_job(
    TranscriptionJobName=job_name,
    Media={'MediaFileUri': "s3://demo-bucket-transcription-course/speech.mp3"},
    MediaFormat="mp3",
    LanguageCode="pt-BR" # https://docs.aws.amazon.com/transcribe/latest/dg/supported-languages.html
)

ConflictException: An error occurred (ConflictException) when calling the StartTranscriptionJob operation: The requested job name already exists. Use a different job name.

We can check in a while loop if our job is finished using *get_transcription_job(TranscriptionJobName)*

In [30]:
import time
while True:
    status = transcribe_client.get_transcription_job(TranscriptionJobName=job_name)
    if status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
        break
    print("Not ready yet...")
    time.sleep(10)
print(status)

{'TranscriptionJob': {'TranscriptionJobName': 'TranscribeJob1', 'TranscriptionJobStatus': 'COMPLETED', 'LanguageCode': 'pt-BR', 'MediaSampleRateHertz': 24000, 'MediaFormat': 'mp3', 'Media': {'MediaFileUri': 's3://demo-bucket-transcription-course/speech.mp3'}, 'Transcript': {'TranscriptFileUri': 'https://s3.us-east-1.amazonaws.com/aws-transcribe-us-east-1-prod/924641445820/TranscribeJob1/f52e9b0f-9cdd-47a7-a6d5-b31701d282b6/asrOutput.json?X-Amz-Security-Token=IQoJb3JpZ2luX2VjEBUaCXVzLWVhc3QtMSJHMEUCIAVAlTIbXvZbLJ29%2BVsruzIjsXVkQjm%2Flww1FuZ%2F08LYAiEArFDl08KJcJom%2F5xzshtXzKTixkk5VuPE5SmDm90qqOkquwUIvf%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FARAEGgwyNzY2NTY0MzMxNTMiDA6Ut259VKiruI9nmyqPBVIUte0c1EzaiCHLq4IDNDawAzoPztr4uWtWDfbsUcbLcRPmEqhR0FJmQwq1mnUm%2FF1D50e1DK%2FpEq%2FQ7A3VCwBpgI%2FfsjzrteOUjlEyv%2FFmortZcReFlNjFeiCErEyLYppN5%2FYSgt8%2FXcd0AvNjcyIeOD3xKJY4ZHOUVS5Uyx7lenSmicj6B53o3NfTQVjx4t49N%2BsIyW5EZrK1QoQwi%2BHOxy%2B%2BCBg2MUGQwo0VagKJx3tenrjiaphsUg5n0HHsItHm5s7F5Jb6gOW1NEosBjemMm%2BHy9pwVVbxq

In [31]:
status['TranscriptionJob']['TranscriptionJobStatus']

'COMPLETED'

Once the transcription job is complete, AWS Transcribe provides a signed URL (usually an S3 URL) from which you can download the transcribed results. The result is in the form of a JSON file. We can also use python to download it.

In [32]:
status['TranscriptionJob']['Transcript']['TranscriptFileUri']

'https://s3.us-east-1.amazonaws.com/aws-transcribe-us-east-1-prod/924641445820/TranscribeJob1/f52e9b0f-9cdd-47a7-a6d5-b31701d282b6/asrOutput.json?X-Amz-Security-Token=IQoJb3JpZ2luX2VjEBUaCXVzLWVhc3QtMSJHMEUCIAVAlTIbXvZbLJ29%2BVsruzIjsXVkQjm%2Flww1FuZ%2F08LYAiEArFDl08KJcJom%2F5xzshtXzKTixkk5VuPE5SmDm90qqOkquwUIvf%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FARAEGgwyNzY2NTY0MzMxNTMiDA6Ut259VKiruI9nmyqPBVIUte0c1EzaiCHLq4IDNDawAzoPztr4uWtWDfbsUcbLcRPmEqhR0FJmQwq1mnUm%2FF1D50e1DK%2FpEq%2FQ7A3VCwBpgI%2FfsjzrteOUjlEyv%2FFmortZcReFlNjFeiCErEyLYppN5%2FYSgt8%2FXcd0AvNjcyIeOD3xKJY4ZHOUVS5Uyx7lenSmicj6B53o3NfTQVjx4t49N%2BsIyW5EZrK1QoQwi%2BHOxy%2B%2BCBg2MUGQwo0VagKJx3tenrjiaphsUg5n0HHsItHm5s7F5Jb6gOW1NEosBjemMm%2BHy9pwVVbxq%2Fs7GzE%2BFGTlqvkz6HjhajCgoCeXUm5nwYCEAZWCy0Xj1f96KEo4W2sFqxsIzcp5hCZD3YDgw6zKW4zra%2BB5UfgXTOuVRTfjc8uuUOQCohW%2FCSIOvYa%2BlYCAhvF8vA9y93rP5pDEoc4iPdhgmKjoVDDClw8btvMnr6O8AQS5Va69f29eqyXnQCNgX9o44%2BJeI5DsaHhf62OZEeVymWlIOrbJyJI3zTLJdIYB2ulcWb0lcewENMsEVeYQx%2B%2FRuKkTAnRrQaws%2Fz5sFMMrF6Nr7n9

In [34]:
import requests
response = requests.get(status['TranscriptionJob']['Transcript']['TranscriptFileUri'])

with open("/home/jovyan/work/Projetos 2024/AWS Transcribe/transcribed.json", 'wb') as file:
    file.write(response.content)

The JSON file output by AWS Transcribe provides detailed information about the transcribed content. Here's a brief overview of its structure:

- **jobName:** 
  - The name of the transcription job
- **accountId:** 
  - Your AWS account ID.
- **results:** 
  - Contains the main transcription data.
  - **transcripts:** 
    - An array with the transcribed text.
  - **items:** 
    - An array containing details about each transcribed word or punctuation.
    - Each item can have:
      - **start_time:** 
        - Start time of the word (if applicable).
      - **end_time:** 
        - End time of the word (if applicable).
      - **alternatives:** 
        - Alternative transcriptions for the word.
      - **type:** 
        - Whether the item is a word or punctuation.
- **status:** 
  - The status of the transcription job (e.g., "COMPLETED").
- **jobCreationDate:** 
  - When the transcription job was created.
- **jobCompletionDate:** 
  - When the transcription job was completed.

Let's load our downloaded file

In [35]:
import json
with open("/home/jovyan/work/Projetos 2024/AWS Transcribe/transcribed.json", "r") as f:
    transcribed_data = json.load(f)

In [37]:
transcribed_data

{'jobName': 'TranscribeJob1',
 'accountId': '924641445820',
 'status': 'COMPLETED',
 'results': {'transcripts': [{'transcript': ''}], 'items': []}}

In [36]:
transcribed_data["results"]

{'transcripts': [{'transcript': ''}], 'items': []}

In [38]:
transcribed_data["results"]["transcripts"]

[{'transcript': ''}]

In [39]:
transcribed_data["results"]["transcripts"][0]

{'transcript': ''}

In [40]:
transcribed_data["results"]["transcripts"][0]["transcript"]

''

Great! That perfectly matches our mp3.

### Delete the job
To delete the job from your history, you can use *client.delete_transcription_job(TranscriptionJobName)*

In [41]:
transcribe_client.delete_transcription_job(TranscriptionJobName="TranscribeJob1")

{'ResponseMetadata': {'RequestId': 'f6b33c55-6ba2-4229-bc15-29f48bd69321',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f6b33c55-6ba2-4229-bc15-29f48bd69321',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Wed, 17 Jan 2024 13:48:42 GMT'},
  'RetryAttempts': 0}}